In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [ ]:
# Load the dataset
data = pd.read_csv('Churn_Modelling.csv')    
data.head()

In [ ]:
# Preprocess the data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

In [ ]:
# Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

In [ ]:
# One-hot encode 'Geography' column
# because it has more than two categories so if we label it as 0 1 2 it may mislead the model to think there is some order like 0<1<2

